
# 1. Preliminares

1.   Cargar el contenedor.

 `docker run --rm -it -v C:\cerdito:/datalake -p 8088:8088 jdvelasq/pig:0.17.0-pseudo`

2.   Obtiene los datos, puedes tenerlos en la carpeta compartida o descargarlos con wget. Requerimos el archivo peliculas.txt

3. Verifica que tienes los archivos en el ambiente local del contenedor con el comando 
`root@algo:/datalake# ls`

4. Ahora verifica que archivos tienes en el sistema de archivos hadoop. 
`
root@algo:/datalake# hadoop fs -ls
`

5. Ahora los datos locales en el sistema de archivos Hadoop
`root@algo:/datalake# hadoop fs -put peliculas.txt`

6. Vuelve a verificar que tienes en el sistema de archivos de Hadoop.

```
root@algo:/datalake# hadoop fs -ls

Found 1 items
-rw-r--r--   1 root supergroup        348 2021-04-17 00:11 peliculas.txt
```

7. Consultemos las dos primeras líneas del archivo que se ha cargado en el sistema de archivos de Hadoop, se combina con el comando `head` de bash.

`root@algo:/datalake# hadoop fs -cat peliculas.txt | head -n 2`

* Recuerda para consultar lo mismo de manera local es con:
`root@algo:/datalake# head -n 2 peliculas.txt`

* Consultar tamaño en disco: `root@algo:/datalake# hadoop fs -df -h` 

* Consultar el último kb de información de un archivo
`root@algo:/datalake# hadoop -tail peliculas.txt` 


8. Ejecutamos PIG con el comando `pig` prestar atención al Command Prompt o Símbolo del Sistema, en este caso `grunt>`
```
root@f3a874f91b1b:/datalake# pig
2021-04-17 00:14:15,156 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
grunt> 
```
Ahora pig está a tus órdenes para recibir comandos. Podemos salir de pig escribiendo el comando `quit.

* Observación: Podemos usar comandos hadoop dentro de pig, por ejemplo, consultar el tamaño en disco: `grunt> fs -df -h`

10. Pig, en modo pseudo (distribuído), trabaja con los archivos almacenados en el sistema de archivos Hadoop. 

`misPelis = LOAD 'peliculas.txt' USING PigStorage(',') as (id,nombre,ano,calificacion,duracion);`

* Debes siempre cerrar o terminar un comando con punto y coma (;) en caso de no hacerlo te aparecerá el siguiente simbolo`>>` 

11. Consultemos el contenido de la bolsa (bag). Los nombres de bolsas son sensibles a las mayúsculas. `grunt> dump misPelis;`

12. Consultar la estructura de la bolsa. `grunt> describe misPelis;`




# 2. Consultas - Filtrado (FILTER) - haciendo casting

Recuerda las consultas se ejecutan en `grunt>`

¿Notas algo raro en la estructura de la bosa? `grunt> describe misPelis;`

1. Consutar las peliculas con una calificación mayor a 4.1

`misPelisC1 = FILTER misPelis BY (float)calificacion > 4.1;`

`DUMP misPelisC1; --muestra la bolsa`

2. Almacenar la consulta en disco HDSF (Hadoop)fs: 

`STORE misPelisC1 into 'lasMejoresPelis';`

Alternativa indicando el separador: `STORE misPelisC1 INTO 'lasMejoresPelis' USING PigStorage(';');`

Contenido de la carpeta creada: `fs -ls lasMejoresPelis`

Contenido del archivo que contiene los resultados: `fs -cat lasMejoresPelis/part-m-00000`

Copiamos el archivo en la carpeta local:`fs -copyToLocal lasMejoresPelis/part-m-00000`

También se puede copiar toda la carpeta en la carpeta local:`fs -copyToLocal lasMejoresPelis`


# 3. Cargar datos con tipado y filtrado (FILTER).

1. Definir el tipo de datos durante la carga:

`grunt> misPelisData = LOAD 'peliculas.txt' USING PigStorage(',') as (id:int,nombre:chararray,ano:int,calificacion:double,duracion:int);`

Verificamos que cargaron: `dump misPelisData;`

Verificamos la estructura de la bolsa: `describe misPelisData;`

* ¿Qué sucede si se realiza un cambio en el archivo local de datos y deseo reflejarlos en la bolsa?

2. Consultar las peliculas entre 1990 y 1998

`pelisViejas = FILTER misPelisData by ano > 1989 and ano < 1995;`

`dump pelisViejas;`

3. Peliculas que comienzan por la letra L

`pelisL = FILTER misPelisData by nombre matches 'L.*';`

`dump pelisL;`

4. Peliculas con una duración de más de dos horas (7200 segundos)

`largometrajes = FILTER misPelisData by duracion >= 7200;`

`dump largometrajes;`








# 4. Transformación de columnas (FOREACH)

`duracionHoras = FOREACH misPelisData GENERATE nombre, (double)(duracion/3600);`

`dump duracionHoras;`

El resultado genera números enteros, porque división de enteros es resultado entero.

`duracionHoras = FOREACH misPelisData GENERATE nombre, (double)duracion/3600;`

`dump duracionHoras;`

Ahora redondeamos a dos cifras decimales;

`duracionHoras = FOREACH misPelisData GENERATE nombre, ROUND_TO((double)duracion/3600, 2);`

`dump duracionHoras;`

***Ejercicio***, almacena este resultado en el disco local.

[Ver más funciones de pig](https://www.tutorialspoint.com/apache_pig/apache_pig_math_functions.htm)

# 5. Agrupaciones (GROUP)

1. Agrupamos las tupas "registros" por año de la pelicula.

`gPorAno = GROUP misPelisData by ano;`

`DUMP gPorAno;`

2. Cantidad de Peliculas por año.

`conteoPorAno = FOREACH gPorAno GENERATE group, COUNT(misPelisData)`

`dump conteoPorAno;`

***Ejercicio***: Contar la cantidad de películas por año antes de 1995.

# 6. Ordenamiento ORDER y LIMIT

Ordenar descendentemente las peliculas por año.

`ordenDesc = ORDER misPelisData by ano DESC;`

`dump ordenDesc;`

*   Se ordena ascendentemente con `ASC`

Ahora mostramos los 3 primeros peliculas ordenadas por año descendentemente.

`misTres = LIMIT ordenDesc 3;`

`dump misTres;`




# Recomendados


[Wikitechy](https://www.wikitechy.com/tutorials/apache-pig/apache-pig-example)

[Tutorialspoint](https://www.tutorialspoint.com/apache_pig/apache_pig_architecture.htm)